In [1]:
# \begin{figure}[ht]
#   \subfloat[boopoety]{
# 	\begin{minipage}[c][1\width]{
# 	   0.3\textwidth}
# 	   \centering
# 	   \includegraphics[width=\textwidth]{/Users/kirstenziman/Documents/github/attention-memory-paper/supp_figures/sustain1violin.pdf}
# 	\end{minipage}}
#  \hfill 	
#   \subfloat[Replication cohort]{
# 	\begin{minipage}[c][1\width]{
# 	   0.3\textwidth}
# 	   \centering
# 	   \includegraphics[width=\textwidth]{/Users/kirstenziman/Documents/github/attention-memory-paper/supp_figures/sustain2violin.pdf}
# 	\end{minipage}}
#  \hfill	
#   \subfloat[Combined data]{
# 	\begin{minipage}[c][1\width]{
# 	   0.3\textwidth}
# 	   \centering
# 	   \includegraphics[width=\textwidth]{example-image-a.pdf}
# 	\end{minipage}}
# \caption{\textbf{Sustained Attention Experiment.}}
# \end{figure}

In [2]:
import os

In [14]:
os.listdir('~/Documents')


FileNotFoundError: [Errno 2] No such file or directory: '~/Documents'